In [1]:
import os
import yaml
import copy
import time
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

### Tmean, Tmax, Tmean, TPmean, TPmax

In [3]:
save_dir = f'/glade/derecho/scratch/ksha/EPRI_data/METRICS_GLOBE/'
base_dir = '/glade/derecho/scratch/ksha/EPRI_data/ERA5_grid/'

In [4]:
list_ds = []

var_map = {
    '2m_temperature': "TREFHT",
    'maximum_2m_temperature_since_previous_post_processing': "TREFHTMX",
    'total_precipitation': "PRECT",
}

for year in range(1958, 2026):

    # get data and variable
    fn_ERA5 = base_dir + f'ERA5_{year}.zarr'
    # subset needed vars
    ds = xr.open_zarr(fn_ERA5, consolidated=True)[list(var_map)].rename(var_map)
    
    # reductions (file is already one year)
    ds_max_TP = ds[["PRECT"]].max("time", skipna=True).rename({"PRECT": "PRECT_max"})
    ds_mean_TP = ds[["PRECT"]].mean("time", skipna=True).rename({"PRECT": "PRECT_mean"})
    ds_max_TMX = ds[["TREFHTMX"]].max("time", skipna=True).rename({"TREFHTMX": "TREFHTMX_max"})
    ds_mean_TMEAN = ds[["TREFHT"]].mean("time", skipna=True).rename({"TREFHT": "TREFHT_mean"})
    
    ds_merge = xr.merge([ds_max_TP, ds_mean_TP, ds_max_TMX, ds_mean_TMEAN])
    list_ds.append(ds_merge)

ds_all = xr.concat(list_ds, dim='year')
ds_all = ds_all.assign_coords({'year': np.arange(1958, 2026)})
ds_all = ds_all.chunk({'year': 68, 'lat': 192, 'lon': 288})

In [5]:
save_name = save_dir + 'ERA5_minmax.zarr'
# ds_all.to_zarr(save_name, mode='w')
print(save_name)

/glade/derecho/scratch/ksha/EPRI_data/METRICS_GLOBE/ERA5_minmax.zarr


### Detrend

In [ ]:
def detrend_linear(da, dim="time"):
    t = xr.DataArray(np.arange(da.sizes[dim]), dims=dim, coords={dim: da[dim]})

    valid = np.isfinite(da)
    t_valid = t.where(valid)
    da_valid = da.where(valid)

    t_mean = t_valid.mean(dim, skipna=True)
    y_mean = da_valid.mean(dim, skipna=True)

    cov = ((t_valid - t_mean) * (da_valid - y_mean)).mean(dim, skipna=True)
    var = ((t_valid - t_mean) ** 2).mean(dim, skipna=True)

    slope = cov / var
    intercept = y_mean - slope * t_mean

    trend = slope * t + intercept
    return da - trend

In [16]:
fn = '/glade/derecho/scratch/ksha/EPRI_data/METRICS_GLOBE/ERA5_minmax.zarr'
ds_ERA5 = xr.open_zarr(fn)

year_valid = np.arange(1968, 2021)
ds_ERA5 = ds_ERA5.sel(year=year_valid)

ds_ERA5_detrend = ds_ERA5 #.copy
for v in ds_ERA5.data_vars:
    ds_ERA5_detrend[v] = detrend_linear(ds_ERA5[v], dim="year")

ds_ERA5_detrend = ds_ERA5_detrend.rename({v: f"{v}_detrend" for v in ds_ERA5_detrend.data_vars})

In [18]:
save_name = save_dir + 'ERA5_minmax_detrend.zarr'
# ds_ERA5_detrend.to_zarr(save_name, mode='w')
print(save_name)

/glade/derecho/scratch/ksha/EPRI_data/METRICS_GLOBE/ERA5_minmax_detrend.zarr
